# Solving the FEM equations for an extending ice cube with SymPy

## Intro, imports and images
This script solves the system of PDEs describing a cube of ice, of side length $h$, spreading horizontally and compressing vertically under its own weight. The setup is depicted in Figure 1 of the main text, which is reproduced [below](#Figure-1). This script uses the symbolic math library ```SymPy``` (v. 1.11.1) to solve the FEM equations symbolically, and it provides the option of solving numerically with ```SciPy```. The workflow of the code mirrors the workflow beginning on page 15 of the paper and can be used to verify the derivations presented in that section.

In [1]:
import sympy as sp
from scipy.integrate import dblquad, tplquad #double and triple integration

### Figure 1

The domain over which we implement our finite element system of equations. The shaded area represents the quadrant of the ice block within which we obtain our 3D velocity field. The block is meshed as indicated by the node numbers shown, with node 1 at the origin, node 2 at $(0, h, 0)$, and so on. The bottom surface ($z = 0$) rests on a hard barrier, while the surfaces at $x = h$, $y = h$, and $z = h$ are unobstructed. 

<img src='figures/cube.PNG' width='500'>

This code will be slightly more flexible than the example presented in the main text. The default setting will be the situation of biaxial extension (i.e., extending both longitudinally and laterally), as in the manuscript. However, this script will also provide the option of solving for uniaxial extension (i.e., extending only longitudinally, between slippery lateral barriers). Setting ```uniaxial_spread``` to ```False``` in the code block below will solve the biaxial extension problem posited in the manuscript. Setting ```uniaxial_spread``` to ```True``` will solve the uniaxial extension problem.

The script can be run to provide symbolic solutions (in which all variables are left general) by setting ```symbolic_solution``` to ```True```. Otherwise, if set to ```False```, the equations are solved numerically, given the values specified below. 

In [2]:
uniaxial_spread = False
symbolic_solution = True

## Spatial variables, dimensions, and physical parameters

Define the spatial coordinates $x$, $y$, and $z$, alongside the side length $h$, viscosity $\mu$, density $\rho$, gravitational constant $g$, rate factor $A$, and flow exponent $n$. Units are meters, seconds, kilograms, Pascals, etc. 

As well, define the lithostatic vector $\boldsymbol{\Lambda}$ and body force vector $\mathbf{b}$, which are given implicitly in Equation 8 as $\boldsymbol{\Lambda} := \rho g(h-z)\begin{bmatrix}1 & 1 & 1 & 0 & 0 & 0\end{bmatrix}^T$ and $\mathbf{b} := -\rho g\begin{bmatrix}0 & 0 & 1\end{bmatrix}^T$. 

In [3]:
x, y, z = sp.symbols('x, y, z', real=True)
n_nodes = 8 #one node at each corner of the cube 

h = 100
μ, ρ, g, A, n = 4e13, 917, 9.8, 1e-23, 3

if symbolic_solution: #if we want a symbolic solution:
    h = sp.symbols('h', positive=True, constant=True) #keep these symbols general
    μ, ρ, g, A, n = sp.symbols('μ, ρ, g, A, n', positive=True, constant=True)
    
Λ = ρ*g*(h-z)*sp.Matrix((1, 1, 1, 0, 0, 0))
b = -ρ*g*sp.Matrix((0, 0, 1))

## Symmetric and resistive gradient operators 

The symmetric gradient operator and resistive gradient operators are defined

$\boldsymbol{\nabla}_S := \begin{bmatrix}\frac{\partial}{\partial x} & 0 & 0\\
0 & \frac{\partial}{\partial y} & 0\\
0 & 0 & \frac{\partial}{\partial z}\\
\frac{\partial}{\partial y} & \frac{\partial}{\partial x} & 0\\
\frac{\partial}{\partial z} & 0 & \frac{\partial}{\partial x}\\
0 & \frac{\partial}{\partial z} & \frac{\partial}{\partial y}
\end{bmatrix} \hspace{1cm}$ and $\hspace {1cm}\boldsymbol{\nabla}_R := \begin{bmatrix}4\frac{\partial}{\partial x} & 2\frac{\partial}{\partial y} & 0\\
2\frac{\partial}{\partial x} & 4\frac{\partial}{\partial y} & 0\\
2\frac{\partial}{\partial x} & 2\frac{\partial}{\partial y} & 2\frac{\partial}{\partial z}\\
\frac{\partial}{\partial y} & \frac{\partial}{\partial x} & 0\\
\frac{\partial}{\partial z} & 0 & \frac{\partial}{\partial x}\\
0 & \frac{\partial}{\partial z} & \frac{\partial}{\partial y}
\end{bmatrix}$

We include these as functions, ```nabla_S``` and ```nabla_R```. For example, the function ```nabla_S``` will take a matrix ```M``` as input and return the matrix product $\boldsymbol{\nabla}_S\mathbf{M}$. 

In [4]:
def nabla_S(M):
    out = sp.zeros(6, M.cols) #initialize output
    for i in range(M.cols):
        out[0, i] = sp.diff(M[0, i], x) #∂/∂x of entry at (0, i)
        out[1, i] = sp.diff(M[1, i], y) #∂/∂y of entry at (1, i)
        out[2, i] = sp.diff(M[2, i], z) #etc.
        out[3, i] = sp.diff(M[0, i], y) + sp.diff(M[1, i], x)
        out[4, i] = sp.diff(M[0, i], z) + sp.diff(M[2, i], x)
        out[5, i] = sp.diff(M[1, i], z) + sp.diff(M[2, i], y)
    return out #return the dot product ∇_S*M

def nabla_R(M):
    out = sp.zeros(6, M.cols) #initialize output
    for i in range(M.cols):
        out[0, i] = 4*sp.diff(M[0, i], x) + 2*sp.diff(M[1, i], y) #4∂/∂x +2∂/∂y of entry at (0, i)
        out[1, i] = 2*sp.diff(M[0, i], x) + 4*sp.diff(M[1, i], y) #etc.
        out[2, i] = 2*sp.diff(M[0, i], x) + 2*sp.diff(M[1, i], y) + 2*sp.diff(M[2, i], z)
        out[3, i] = sp.diff(M[0, i], y) + sp.diff(M[1, i], x)
        out[4, i] = sp.diff(M[0, i], z) + sp.diff(M[2, i], x)
        out[5, i] = sp.diff(M[1, i], z) + sp.diff(M[2, i], y)
    return out #return the dot product ∇_R*M


## Shape functions and their gradients

The nodal shape functions $N_1$ through $N_8$ are defined such that $N_i$ evaluates to one at node $i$ and zero at every other node. It can be directly verified that the functions below satisfy this property for the setup shown in Figure 1. These functions are also given in Equation 26.

In [5]:
N1 = (h-x)*(h-y)*(h-z)/h**3
N2 = (h-x)*y*(h-z)/h**3
N3 = x*y*(h-z)/h**3
N4 = x*(h-y)*(h-z)/h**3
N5 = x*(h-y)*z/h**3
N6 = (h-x)*(h-y)*z/h**3
N7 = (h-x)*y*z/h**3
N8 = x*y*z/h**3

#For example, node 2 is at (0, h, 0) and node 5 is at (h, 0, h)
print('N2 evaluated at node 2:', 'N2(0, h, 0) =', N2.subs({x:0, y:h, z:0}))
print('N2 evaluated at node 5:', 'N2(h, 0, h) =', N2.subs({x:h, y:0, z:h}))

N2 evaluated at node 2: N2(0, h, 0) = 1
N2 evaluated at node 5: N2(h, 0, h) = 0


Next, it's useful to express these shape function in matrix form, defining 

$\mathbf{N} = \begin{bmatrix}N_1 & 0 & 0 & N_2 & 0 & 0 & ... & N_8 & 0 & 0\\
0 & N_1 & 0 & 0 & N_2 & 0 & ... & 0 & N_8 & 0\\
0 & 0 & N_1 & 0 & 0 & N_2 & ... & 0 & 0 & N_8
\end{bmatrix}$

This is coded as ```mtx_N``` below. 

In [6]:
#First, write the shape functions as an 8-by-1 vector
vec_N = sp.Array((N1, N2, N3, N4, N5, N6, N7, N8))

#Now construct the shape function as a 3-by-24 matrix
I = sp.eye(3) #3x3 identity matrix
mtx_N = sp.zeros(3, 3*n_nodes) #initialize the matrix N
for i in range(n_nodes):
    mtx_N[0:3, 3*i:3*i+3] = I*vec_N[i] #each ith 3x3 submatrix gets assigned I*N_i

#Output a visual representation of mtx_N:
print('Shape function in matrix form:')
mtx_N

Shape function in matrix form:


Matrix([
[(h - x)*(h - y)*(h - z)/h**3,                            0,                            0, y*(h - x)*(h - z)/h**3,                      0,                      0, x*y*(h - z)/h**3,                0,                0, x*(h - y)*(h - z)/h**3,                      0,                      0, x*z*(h - y)/h**3,                0,                0, z*(h - x)*(h - y)/h**3,                      0,                      0, y*z*(h - x)/h**3,                0,                0, x*y*z/h**3,          0,          0],
[                           0, (h - x)*(h - y)*(h - z)/h**3,                            0,                      0, y*(h - x)*(h - z)/h**3,                      0,                0, x*y*(h - z)/h**3,                0,                      0, x*(h - y)*(h - z)/h**3,                      0,                0, x*z*(h - y)/h**3,                0,                      0, z*(h - x)*(h - y)/h**3,                      0,                0, y*z*(h - x)/h**3,                0,          0, x*y*

The gradients of the shape function are $\mathbf{B}_S = \boldsymbol{\nabla}_S\mathbf{N}$ and $\mathbf{B}_R = \boldsymbol{\nabla}_R\mathbf{N}$, which are evaluated as

In [7]:
B_S = nabla_S(mtx_N)
B_R = nabla_R(mtx_N)

## Boundary conditions

### Velocity (dirichlet) boundary conditions

The vector of nodal velocities, $\mathbf{d}$, is given by 

$\mathbf{d} = [u_{1x}, u_{1y}, u_{1z}, u_{2x}, u_{2y}, u_{2z}, ..., u_{8x}, u_{8y}, u_{8z}]^T$.

Many of these components are zero by nature of the setup (see the paragraph following Equation 29). For example, $u_{1z} = u_{2z} = u_{3z} = u_{4z} = 0$ because the bottom of the cube rests on a hard barrier. In the biaxial spreading case, by symmetry, $u_x = 0$ on the $y$ axis and $u_y = 0$ on the $y$ axis, and so $u_{1x} = u_{2x} = u_{6x} = u_{7x} = u_{1y} = u_{4y} = u_{5y} = u_{6y} = 0$. 

To code this information, we define a list, ```BCs```, having the same length as $\mathbf{d}$ but consisting only of 1s and 0s. ```BCs``` will have 0s wherever velocity is prescribed to be zero, and 1s wherever velocity is unknown. In the biaxial spreading case, we have:

In [8]:
BCs = [0, 0, 0, #u_1x, u_1y, u_1z = 0
     0, 1, 0, #u_2x, u_2z = 0 while u_2y is unknown
     1, 1, 0, #u_3x, u_3y are unknown while u_3z is zero
     1, 0, 0, #etc.
     1, 0, 1, 
     0, 0, 1, 
     0, 1, 1,
     1, 1, 1]

If we are solving the uniaxial extension problem, then we should additionally set every $u_y$ component to zero:

In [9]:
if uniaxial_spread:
    for i in range(len(BCs)):
        if i%3 == 1: #for u_1y, u_2y, u_3y, etc.:
            BCs[i] = 0 #set this component to zero

### Traction (Neumann) boundary conditions

In the biaxial spreading case discussed in the manuscript, the positive $x$ and $y$ faces are associated with depth-averaged ice cliff boundary conditions:

$\mathbf{t}_X = \rho g\left(z - \frac{h}{2}\right)\hat{\mathbf{i}}\hspace{1cm}$ and $\hspace{1cm} \mathbf{t}_Y = \rho g\left(z - \frac{h}{2}\right)\hat{\mathbf{j}}$

For consistency with the uniaxial case, we can also include a placeholder boundary condition at the negative $Y$ face, setting $\mathbf{t}_{negY} = \mathbf{0}$.

In [10]:
t_X = ρ*g*(z - h/2)*sp.Matrix((1, 0, 0)) #t_X = σ_xx|_{x=h}*i
t_Y = ρ*g*(z - h/2)*sp.Matrix((0, 1, 0)) #t_Y = σ_yy|_{y=h}*j
t_negY = sp.Matrix((0, 0, 0)) #placeholder value #t_negY = [0, 0, 0]

In the uniaxial case, the positive $x$ boundary remains a free ice cliff, but the positive and negative $y$ boundaries must have zero lateral extension; that is, $\tau_{yy}|_{y = h} = \tau_{yy}|_{y = 0} = 0$. This is the case when the net stress, $\sigma_{yy}$, is simply equal to the overburden pressure at these locations. Therefore, in a uniaxial extending regime, $\mathbf{t}_{Y} = \rho g(z-h)\hat{\mathbf{j}}$ and $\mathbf{t}_{negY} = \rho g(z-h)\left(-\hat{\mathbf{j}}\right)$.

In [11]:
if uniaxial_spread:
    t_Y = ρ*g*(z - h)*sp.Matrix((0, 1, 0)) #t_Y = σ_yy|_{y=h}*j
    t_negY = ρ*g*(z - h)*sp.Matrix((0, -1, 0)) #t_negY = σ_yy|_{y=0}*-j

## Stiffness matrix and force vector

We evaluate the stiffness matrix $\mathbf{K}$ as the integral $\iiint_{x,y,z = 0}^h\mathbf{B}_S^T\mu\mathbf{B}_Rdxdydz$ (see Eq. 33). The symbolic solution will be calculated with SymPy's ```integrate``` function, and the numerical solution will be calculated using SciPy's ```tplquad``` function. 

Because ```sympy.integrate``` is somewhat slow (it takes about 20 seconds to compute all of $\mathbf{K}$'s $24^2$ triple integrals symbolically), we'll calculate only the entries of $\mathbf{K}$ which are needed. Any entry of $\mathbf{K}$ corresponding to a dirichlet boundary condition will not influence the solution step, so we'll leave these out (this cuts the symbolic calculation down to about 4 seconds, but it won't make a noticeable difference if solving numerically with ```scipi.integrate.tplquad```).  

In [12]:
K = sp.zeros(3*n_nodes, 3*n_nodes) #initialize K as a 24x24 matrix of zeros
integrand_K = B_S.T*μ*B_R #we'll populate K by integrating select entries of this 24x24 matrix

for i in range(K.rows): #for each row:
    for j in range(K.cols): #and each column:
        if BCs[i] != 0 and BCs[j] != 0: #if NEITHER the row nor the column correspond to a velocity BC:
            integrand = integrand_K[i, j] #we'll integrate this entry using one of the methods below.
            if symbolic_solution: #if we want a symbolic solution, we'll integrate symbolically:
                K[i, j] =  sp.integrate(sp.integrate(sp.integrate(integrand, (x, 0, h)), (y, 0, h)), (z, 0, h))
            else: #otherwise, if we just want a numerical solution:
                integrand = sp.lambdify((x, y, z), integrand) #interpret integrand as a function
                K[i, j] = tplquad(integrand, 0, h, 0, h, 0, h)[0] #and integrate numerically with scipy
        #elif the entry DOES correspond to a velocity BC:
            #leave it as zero.

The force vector can be seperated into components $\mathbf{f} = \mathbf{f}_\Gamma + \mathbf{f}_\Omega + \mathbf{f}_\Lambda$, each of which is calculated similarly to $\mathbf{K}$ (see Eq. 16 for the general theory, and Eq. 34 for application to the biaxial extension problem).

In [13]:
f_Γx, f_Γy, f_Γnegy = sp.zeros(3*n_nodes, 1), sp.zeros(3*n_nodes, 1), sp.zeros(3*n_nodes, 1)
f_Ω, f_Λ = sp.zeros(3*n_nodes, 1), sp.zeros(3*n_nodes, 1)
    
integrand_Γx = mtx_N.T.subs(x, h)*t_X #integrand_Γx will be integrated in calculating f_Γx
integrand_Γy = mtx_N.T.subs(y, h)*t_Y #for calculating f_Γy
integrand_Γnegy = mtx_N.T.subs(y, 0)*t_negY #for calculating f_Γnegy
integrand_Ω = mtx_N.T*b #for calculating f_Ω
integrand_Λ = B_S.T*Λ #for calculating f_Λ

for i in range(3*n_nodes):
    if symbolic_solution:
        f_Γx[i] = sp.integrate(sp.integrate(integrand_Γx[i], (y, 0, h)), (z, 0, h)) #term 1 of Eq. 34
        f_Γy[i] = sp.integrate(sp.integrate(integrand_Γy[i], (x, 0, h)), (z, 0, h)) #term 2 of Eq. 34
        f_Γnegy[i] = sp.integrate(sp.integrate(integrand_Γnegy[i], (x, 0, h)), (z, 0, h)) #nonzero only if uniaxial
        f_Ω[i] = sp.integrate(sp.integrate(sp.integrate(integrand_Ω[i], (x, 0, h)), (y, 0, h)), (z, 0, h)) #term 3
        f_Λ[i] = sp.integrate(sp.integrate(sp.integrate(integrand_Λ[i], (x, 0, h)), (y, 0, h)), (z, 0, h)) #term 4
    else:        
        f_Γx[i] = dblquad(sp.lambdify((y, z), integrand_Γx[i]), 0, h, 0, h)[0]
        f_Γy[i] = dblquad(sp.lambdify((x, z), integrand_Γy[i]), 0, h, 0, h)[0]
        f_Γnegy[i] = dblquad(sp.lambdify((x, z), integrand_Γnegy[i]), 0, h, 0, h)[0]
        f_Ω[i] = tplquad(sp.lambdify((x, y, z), integrand_Ω[i]), 0, h, 0, h, 0, h)[0]
        f_Λ[i] = tplquad(sp.lambdify((x, y, z), integrand_Λ[i]), 0, h, 0, h, 0, h)[0]
        
f_Γ = f_Γx + f_Γy + f_Γnegy 
f = f_Γ + f_Ω + f_Λ #total force vector 

## Swapping and partitioning

In order to partition the finite element system of equations into the form shown in Equation 17, it is necessary to systematically sort $\mathbf{K}$, $\mathbf{d}$, and $\mathbf{f}$ (as in Equation 17). The goal is to rearrange $\mathbf{d}$ so that the velocity components chosen as boundary conditions appear as the top entries, with the unknown velocity components at the bottom $-$ while simultaneously sorting $\mathbf{K}$ and $\mathbf{f}$ so as to preserve the system of linear equations represented by $\mathbf{K}\mathbf{d} = \mathbf{f}$. 

Bookkeeping will be done via a list, ```order```, which will document the new placement of each component of $\mathbf{d}$ after sorting.  

In [14]:
order = sorted(range(3*n_nodes), key = lambda i : BCs[i]) 
print('This is the new placement of each velocity component after sorting:')
sp.Array(order)

This is the new placement of each velocity component after sorting:


[0, 1, 2, 3, 5, 8, 10, 11, 13, 15, 16, 18, 4, 6, 7, 9, 12, 14, 17, 19, 20, 21, 22, 23]

For example, in the biaxial extension case (i.e., if ```uniaxial_spread == True```), the 0th through 3rd velocity components have remained in their current positions; the 4th component now appears in position 12; the 5th component has moved to position 4, the 6th to position 13, and so on. (Compare this with the initial placement of velocity boundary conditions [above](#Velocity-(dirichlet)-boundary-conditions).)

Next, we'll use this bookkeeper list to help rearrange the elements of $\mathbf{K}$ and $\mathbf{f}$. 

In [15]:
#Initialize the element-swapped versions of f and K
K_swapped = sp.zeros(K.rows, K.cols)
f_swapped = sp.zeros(f.rows, f.cols)

#And fill the matrices with their appropriate values:
for i in range(K.rows):
    f_swapped[i] = f[order[i]] #swap the correct rows of f
    for j in range(K.cols):
        K_swapped[i, j] = K[order[i], order[j]] 
        #and swap the correct rows AND columns of K

With reference to Equation 17, we now compute $\mathbf{K}_F$ and $\mathbf{f}_F$:

In [16]:
#To partition, first count the number of velocity BCs
VBCs = BCs.count(0) #This is the number of 0s in the previously-defined list, BCs

#Kf and ff are shown in Equation 17
ff = sp.Matrix(f_swapped[VBCs:])
Kf =  K_swapped[VBCs:, VBCs:]

## Computing the linear rheology solution

### Nodal solution

The nodal solution is obtained via Equation 18, with the added simplification that $\mathbf{d}_E$ is just the zero vector (because every velocity boundary condition is zero). We could solve this by writing 

```
df = Kf.inv()*ff
```

However, in general, it is more effecient to solve systems of linear equations without performing matrix inversions. Instead, use SymPy's ```linsolve``` function:

In [17]:
system = Kf, ff
df = sp.linsolve(system) #solves the system Kf*X = ff for unknown X

The complete (but still [sorted](#Swapping-and-partitioning)!) list of nodal velocities is now given as the vector $\begin{bmatrix} \mathbf{d}_E & \mathbf{d}_F \end{bmatrix}^T$:

In [18]:
de = [0]*VBCs 
df = list(list(df)[0])
d_swapped = de + df

Put the elements of ```d_sorted``` back into their original places:

In [19]:
d = [0]*3*n_nodes #initialize the final nodal solution
for i in range(len(d_swapped)):
    d[order[i]] = d_swapped[i] 
d = sp.Matrix(d) #solution is now in the order [u_1x, u_1y, u_1z, ..., u_8x, u_8y, u_8z]

print('This is the (transpose of) the nodal velocity solution, in the correct order:')
d.T

This is the (transpose of) the nodal velocity solution, in the correct order:


Matrix([[0, 0, 0, 0, g*h**2*ρ/(12*μ), 0, g*h**2*ρ/(12*μ), g*h**2*ρ/(12*μ), 0, g*h**2*ρ/(12*μ), 0, 0, g*h**2*ρ/(12*μ), 0, -g*h**2*ρ/(6*μ), 0, 0, -g*h**2*ρ/(6*μ), 0, g*h**2*ρ/(12*μ), -g*h**2*ρ/(6*μ), g*h**2*ρ/(12*μ), g*h**2*ρ/(12*μ), -g*h**2*ρ/(6*μ)]])

### Continuous solution

This nodal solution can be used to obtain the *continuous* velocity solution, $\mathbf{u}$, by interpolation. This is done, by approximation, using the element shape function matrix, with $\mathbf{u} \approx \mathbf{N}\mathbf{d}$.

In [20]:
u = mtx_N*d
u = sp.simplify(u) #group like terms to express compactly

if not symbolic_solution: #if solving numerically:
    def round_expr(expr): #get rid of any annoying coefficients < 1e-20
        return expr.xreplace({n : round(n, 20) for n in expr.atoms(sp.Number)})
    u = round_expr(u)

print('Linear, continuous velocity solution for [u_x, u_y, u_z]:')
u.T

Linear, continuous velocity solution for [u_x, u_y, u_z]:


Matrix([[g*h*x*ρ/(12*μ), g*h*y*ρ/(12*μ), -g*h*z*ρ/(6*μ)]])

## Postprocessing

The next goal will be to compute the nonlinear rheology solution. However, before this can be done, it is necessary to evaluate the stress field and update the effective viscosity. 

Our definition of the operator $\boldsymbol{\nabla}_R$ was prompted by the observation that $\mathbf{R}:= \mu\boldsymbol{\nabla}_R\mathbf{u}$ represents the resistive stresses. Resistive stresses, in turn, can be used to calculate the deviatoric stresses via Equation 20; once deviatoric stresses are known, the effective stress is obtained via Equation 22, and, subsequently, the effective viscosity can be updated via Equation 21. 

In [21]:
#Calculate the resistive stress vector:
R = μ*nabla_R(u) #[R_xx, R_yy, R_zz, R_xy, R_xz, R_yz]^T

#And the associated deviatoric stresses (Eq. 20)
τ = sp.Matrix([sp.Rational(2,3)*R[0] - sp.Rational(1,3)*R[1], #τ_xx
                sp.Rational(2,3)*R[1] - sp.Rational(1,3)*R[0], #τ_yy
                R[2] - sp.Rational(1,3)*(R[0] + R[1]), #τ_zz
                R[3], R[4], R[5]]) #τ_xy, τ_xz, τ_yz

#Calculate the effective stress τ_E (Eq. 22)
expr = sum([τ[i]**2 for i in range(len(τ))])
τ_E = sp.sqrt(sp.Rational(1,2)*expr)

#Update the effective viscosity (Eq. 21)
μ_2 = 1/(2*A*τ_E**(n-1))

print('deviatoric stresses [τ_xx, τ_yy, τ_zz, τ_xy, τ_xz, τ_yz]:')
τ.T

deviatoric stresses [τ_xx, τ_yy, τ_zz, τ_xy, τ_xz, τ_yz]:


Matrix([[g*h*ρ/6, g*h*ρ/6, -g*h*ρ/3, 0, 0, 0]])

## Nonlinear rheology solution

In general, to compute the nonlinear rheology solution, it would be necessary to run the whole scrip again with ```μ_2``` in place of ```μ```. But in this case, we can verify that ```μ_2``` is just a function of $\rho$, $g$, $h$, and $A$, all of which were assumed constant:

In [22]:
μ_2

(sqrt(3)*g*h*ρ/6)**(1 - n)/(2*A)

Therefore, ```μ_2``` is a constant, like ```μ```, and a second iteration would just return the same solution with ```μ_2``` in place of ```μ```. In this case, we may as well update the solution manually:

### Nonlineal nodal solution

In [23]:
if symbolic_solution:
    d_2 = d.subs(μ, μ_2) #substitute μ_2 in place of μ
    d_2 = sp.simplify(d_2) #and simplify
    
else:
    d_2 = d*μ/μ_2

print('Nonlinear nodal velocity solution for [u_1x, u_1y, u1_z, ..., u_8x, u_8y, u_8z]:')
d_2.T

Nonlinear nodal velocity solution for [u_1x, u_1y, u1_z, ..., u_8x, u_8y, u_8z]:


Matrix([[0, 0, 0, 0, A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, 0, A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, 0, A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, 0, 0, A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, 0, -2*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, 0, 0, -2*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, 0, A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, -2*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, -2*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n]])

### Nonlinear continuous solution

In [24]:
if symbolic_solution:
    u_2 = u.subs(μ, μ_2) #substitute μ_2 in place of μ
    u_2 = sp.simplify(u_2) #and simplify
    
else:
    u_2 = u*μ/μ_2

print('Nonlinear, continuous velocity solution [u_x, u_y, u_z]:')
u_2.T

Nonlinear, continuous velocity solution [u_x, u_y, u_z]:


Matrix([[A*g*h*x*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, A*g*h*y*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n, -2*A*g*h*z*ρ*(sqrt(3)*g*h*ρ)**(n - 1)/6**n]])